## Notebook for LSTM Model Creation and Training

In [1]:
# importing the all the required libraries for text preprocessing and model creation

import pandas as pd
import numpy as np
import warnings
import pickle
warnings. filterwarnings("ignore")


from tensorflow.keras.layers import (Embedding,
                         Dense,
                         LSTM,
                         Bidirectional,
                         Dropout
                         )
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# importing the dataset

df = pd.read_csv("Data/lyrics_data.csv")

In [3]:
# droping NaN columns if any.

df.dropna(inplace = True)

In [4]:
df.head()

,Lyrics
0,ennilerinju thudangia thee kanal nenjinakath p...
1,veripidichutti kadal ilaki varunathu pole mara...
2,kathakil mutti vilichath pole bhaamath maanju ...
3,pahuthiluude thalarukilloru shakalavum ini pat...
4,vechaaudhathinte moorchaundente vakkilum innu ...


In [5]:
# tokenizing using keras tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["Lyrics"])

In [6]:
# total vocabulary size
VOC_SIZE = len(tokenizer.word_index)+1
VOC_SIZE

840

In [7]:
#tokenizer.texts_to_sequences(["ennilerinju thudangia thee kanal nenjinakath"])[0]

[171, 172, 89, 173, 174]

In [8]:
songs = df["Lyrics"]

In [9]:
#Function for input data preperation

def data_prep(songs, tokenizer):
    
    data = []
    for sent in songs:
        text_sequence = tokenizer.texts_to_sequences([sent])[0]
        
        for i in range(1, len(text_sequence)):
            data.append(text_sequence[:i+1])
            
    max_len = max([len(sequence) for sequence in data]) # finding the length of the largest sequence
        
    padded_sequence = pad_sequences(data, maxlen = max_len, padding = "pre") # padding to make all sequence equal
   
    
    return padded_sequence

In [10]:
processed_data = data_prep(songs, tokenizer)

In [11]:
processed_data, processed_data.shape

(array([[  0,   0,   0, ...,   0, 171, 172],
        [  0,   0,   0, ..., 171, 172,  89],
        [  0,   0,   0, ..., 172,  89, 173],
        ...,
        [  0,   0,   0, ...,  35,  39,  86],
        [  0,   0,   0, ...,  39,  86,  87],
        [  0,   0,   0, ...,  86,  87,  88]]),
 (1405, 8))

In [12]:
processed_data[0]

array([  0,   0,   0,   0,   0,   0, 171, 172])

In [13]:
X = processed_data[:,:-1]
y = to_categorical(processed_data[:,-1], num_classes = VOC_SIZE)

In [15]:
#length of a sequence
INPUT_LENGTH = len(X[0])

#function for model creation

def create_model():
    
    LEARNING_RATE = 0.001
    
    model = Sequential()
    model.add(Embedding(VOC_SIZE, 32, input_length = INPUT_LENGTH))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))
    model.add(Dense(VOC_SIZE, activation = "softmax"))
    
    model.compile(optimizer = Adam(LEARNING_RATE),
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])
    
    return model
    
    

In [16]:
model = create_model()

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 32)             26880     
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               49664     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 840)               108360    
Total params: 184,904
Trainable params: 184,904
Non-trainable params: 0
_________________________________________________________________


In [18]:
call_back = EarlyStopping(monitor = "loss", min_delta = 0.001, patience = 5)
EPOCHS = 100
BATCH_SIZE = 32

# training the model
history = model.fit(X,
                    y,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    verbose = 1,
                    callbacks = [call_back])


Epoch 1/100
44/44 [==============================] - 1s 21ms/step - loss: 6.7326 - accuracy: 0.0149
Epoch 2/100
44/44 [==============================] - 1s 17ms/step - loss: 6.6136 - accuracy: 0.0178
Epoch 3/100
44/44 [==============================] - 1s 17ms/step - loss: 6.3110 - accuracy: 0.0164
Epoch 4/100
44/44 [==============================] - 1s 18ms/step - loss: 6.0916 - accuracy: 0.0171ETA: 0s - loss: 6.1
Epoch 5/100
44/44 [==============================] - 1s 18ms/step - loss: 5.9570 - accuracy: 0.0242
Epoch 6/100
44/44 [==============================] - 1s 17ms/step - loss: 5.8161 - accuracy: 0.0263
Epoch 7/100
44/44 [==============================] - 1s 16ms/step - loss: 5.6566 - accuracy: 0.0313
Epoch 8/100
44/44 [==============================] - 1s 15ms/step - loss: 5.4748 - accuracy: 0.0406
Epoch 9/100
44/44 [==============================] - 1s 15ms/step - loss: 5.2748 - accuracy: 0.0527
Epoch 10/100
44/44 [==============================] - 1s 14ms/step - loss: 5.0768

44/44 [==============================] - 1s 12ms/step - loss: 0.4619 - accuracy: 0.9395
Epoch 81/100
44/44 [==============================] - 0s 10ms/step - loss: 0.4344 - accuracy: 0.9544
Epoch 82/100
44/44 [==============================] - 1s 12ms/step - loss: 0.4278 - accuracy: 0.9445
Epoch 83/100
44/44 [==============================] - 1s 12ms/step - loss: 0.4211 - accuracy: 0.9466
Epoch 84/100
44/44 [==============================] - 0s 10ms/step - loss: 0.4179 - accuracy: 0.9466
Epoch 85/100
44/44 [==============================] - 1s 13ms/step - loss: 0.4337 - accuracy: 0.9381
Epoch 86/100
44/44 [==============================] - 1s 12ms/step - loss: 0.4094 - accuracy: 0.9438 0s - loss: 0.4222 - ac
Epoch 87/100
44/44 [==============================] - 1s 13ms/step - loss: 0.3976 - accuracy: 0.9523
Epoch 88/100
44/44 [==============================] - 0s 11ms/step - loss: 0.3756 - accuracy: 0.9452
Epoch 89/100
44/44 [==============================] - 1s 12ms/step - loss: 0.3710

In [19]:
# saving the tokenizer and model for future use

model.save("Models/model.h5")
pickle.dump(tokenizer, open("Models/tokenizer.pkl", "wb"))

In [21]:
first_lyric = input("Enter the first few lyrics : ")
next_words = 40
  
for _ in range(next_words):
    
    tokenized = tokenizer.texts_to_sequences([first_lyric])[0] 
    padded_list = pad_sequences([tokenized], maxlen=INPUT_LENGTH, padding="pre") 
    prediction = model.predict_classes(padded_list, verbose=0)

    for word, index in tokenizer.word_index.items():
        if index == prediction:
            pred_word = word
            break
    first_lyric += " " + pred_word
print("\nPredicted Lyrics : \n")
print(first_lyric)

Enter the first few lyrics : Moha

Predicted Lyrics : 

Moha mundiri vaatia ravu sneha rathiude rasanilavu hridhaaragam chiragil chiragil melle ethu arinjatheille neett akaleakale ninnu mutti thiruvaavaniraavu nee thotturummi polum vaarmukilu ponillanne niraumivide kadavathu lipiai hridhaaragam punnara sanjaaram charuvam ummakalaal neett neett neett neett thotturummi maravichu thudangum nenjin niraum athisaakara
